In [3]:
import zipfile

def unzip_file(zip_filepath, output_directory):
    with zipfile.ZipFile(zip_filepath, 'r') as zip_ref:
        zip_ref.extractall(output_directory)

# Example usage
zip_filepath = 'fairface-img-margin025-trainval.zip'
output_directory = 'fairface-img-margin025-trainval/'
unzip_file(zip_filepath, output_directory)

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Dense, Flatten
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

2023-05-05 16:41:18.441586: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-05 16:41:18.490986: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

def load_labels_and_dataframe(path, train_csv, val_csv):
    # Load the CSV files
    train_df = pd.read_csv(os.path.join(path, train_csv))
    val_df = pd.read_csv(os.path.join(path, val_csv))
    
    # Encode the categorical labels
    age_encoder = LabelEncoder()
    gender_encoder = LabelEncoder()
    race_encoder = LabelEncoder()

    train_df['age'] = age_encoder.fit_transform(train_df['age'])
    train_df['gender'] = gender_encoder.fit_transform(train_df['gender'])
    train_df['race'] = race_encoder.fit_transform(train_df['race'])

    val_df['age'] = age_encoder.transform(val_df['age'])
    val_df['gender'] = gender_encoder.transform(val_df['gender'])
    val_df['race'] = race_encoder.transform(val_df['race'])
    
    return train_df, val_df, age_encoder, gender_encoder, race_encoder

In [3]:
# for all data
def load_data(path, input_shape):
    train_datagen = ImageDataGenerator(rescale=1./255)
    val_datagen = ImageDataGenerator(rescale=1./255)

    train_gen = train_datagen.flow_from_directory(
        os.path.join(path, 'train'),
        target_size=input_shape[:2],
        batch_size=32,
        class_mode='categorical'
    )

    val_gen = val_datagen.flow_from_directory(
        os.path.join(path, 'val'),
        target_size=input_shape[:2],
        batch_size=32,
        class_mode='categorical'
    )

    return train_gen, val_gen


In [4]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def load_images(df, path, input_shape):
    images = []
    for _, row in df.iterrows():
        img_path = os.path.join(path, row['file'])
        img = load_img(img_path, target_size=input_shape[:2])
        img_arr = img_to_array(img) / 255.0
        images.append(img_arr)
    return np.array(images)

def load_sample_data(path, input_shape, train_csv, val_csv, sample_percent=0.1):
    # Load the dataframes and label encoders
    train_df, val_df, age_encoder, gender_encoder, race_encoder = load_labels_and_dataframe(path, 
                                                                                            "fairface_label_"+train_csv, 
                                                                                            "fairface_label_"+val_csv)

    # Take a fraction of the dataset
    train_df = train_df.sample(frac=sample_percent)
    val_df = val_df.sample(frac=sample_percent)

    # Load the images and labels
    train_images = load_images(train_df, path, input_shape)
    train_labels = train_df[["age", "gender", "race"]].values
    val_images = load_images(val_df, path, input_shape)
    val_labels = val_df[["age", "gender", "race"]].values

    return train_images, train_labels, val_images, val_labels, age_encoder, gender_encoder, race_encoder

In [5]:
def create_autoencoder(input_shape):
    input_img = Input(shape=input_shape)

    # Encoder
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)

    # Decoder
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = Model(input_img, decoded)
    return autoencoder

In [6]:
def extract_encoder(autoencoder):
    # Extract the encoder layers from the autoencoder model
    input_img = autoencoder.input
    # Assuming the encoder layers are the first 5 layers
    encoded = autoencoder.layers[5].output

    # Create a new model with the encoder layers
    encoder = Model(input_img, encoded)

    return encoder

In [7]:
def create_classification_model(encoder, num_race_classes, num_age_classes, num_gender_classes):
    # Use the encoder's output as input for the classification model
    encoded_input = encoder.output

    # Add a Flatten layer to convert the feature maps into a 1D vector
    x = Flatten()(encoded_input)

    # Add Dense layers for classification
    x = Dense(256, activation='relu')(x)
    x = Dense(128, activation='relu')(x)

    # Add output layers for each attribute: race, age, and gender
    race_output = Dense(num_race_classes, activation='softmax', name='race_output')(x)
    age_output = Dense(num_age_classes, activation='softmax', name='age_output')(x)
    gender_output = Dense(num_gender_classes, activation='softmax', name='gender_output')(x)

    # Create the classification model
    classification_model = Model(encoder.input, [race_output, age_output, gender_output])

    return classification_model

In [8]:
# data_path = "fairface-img-margin025-trainval/fairface-img-margin025-trainval"
# input_shape = (224, 224, 3)
# # Load the data
# train_images, train_labels, val_images, val_labels, age_encoder, gender_encoder, race_encoder = load_sample_data(
#     data_path, input_shape, "train.csv", "val.csv", sample_percent=0.1
# )

In [9]:
# train_images.shape

## Random Forest Model for Benchmarking

In [10]:
import numpy as np

def predict_in_batches(encoder, data, batch_size):
    num_batches = int(np.ceil(len(data) / batch_size))
    predictions = []

    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(data))
        batch_data = data[start_idx:end_idx]
        batch_predictions = encoder.predict(batch_data)
        predictions.append(batch_predictions)

    return np.concatenate(predictions, axis=0)

In [15]:
import tensorflow as tf

tf.keras.backend.clear_session()

In [17]:
!nvidia-smi

Fri May  5 17:34:25 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A40          Off  | 00000000:51:00.0 Off |                    0 |
|  0%   55C    P0    84W / 300W |   3678MiB / 46068MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

def run_model(data_path):
    input_shape = (224, 224, 3)

    # Load the data
    train_images, train_labels, val_images, val_labels, age_encoder, gender_encoder, race_encoder = load_sample_data(
        data_path, input_shape, "train.csv", "val.csv", sample_percent=0.1
    )

    # Create the autoencoder model
    autoencoder = create_autoencoder(input_shape)
    autoencoder.compile(optimizer='adam', loss='mse')

    # Train the autoencoder
    autoencoder.fit(
        train_images, train_images,
        epochs=2,
        batch_size=4,
        validation_data=(val_images, val_images)
    )

    # Extract the encoder
    encoder = extract_encoder(autoencoder)

    # Set the batch size for predictions
    prediction_batch_size = 32

    # Encode the train and validation images
    train_embeddings = predict_in_batches(encoder, train_images, prediction_batch_size)
    val_embeddings = predict_in_batches(encoder, val_images, prediction_batch_size)

    # Train a classifier on the embeddings
    clf = RandomForestClassifier()
    clf.fit(train_embeddings, train_labels)

    # Predict the labels for the validation set
    val_predictions = clf.predict(val_embeddings)

    # Compute the accuracy for each attribute
    print("Validation accuracy:")
    print("Age:")
    print(classification_report(val_labels[:, 0], val_predictions[:, 0], target_names=age_encoder.classes_))
    print("Gender:")
    print(classification_report(val_labels[:, 1], val_predictions[:, 1], target_names=gender_encoder.classes_))
    print("Race:")
    print(classification_report(val_labels[:, 2], val_predictions[:, 2], target_names=race_encoder.classes_))

    return encoder

In [12]:
data_path = "fairface-img-margin025-trainval/fairface-img-margin025-trainval"
run_model(data_path)

2023-05-05 16:41:53.413699: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 466 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:51:00.0, compute capability: 8.6
2023-05-05 16:42:06.122866: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.86GiB (rounded to 5222719488)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-05-05 16:42:06.122918: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-05-05 16:42:06.122942: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 19, Chunks in use: 19. 4.8KiB allocated for chunks. 4.8KiB in use in bin. 844B client-requested in use in bin.
2023-05-05 16:42:06.122958

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

## Deep Learning Model

In [ ]:
def run_model_dl(data_path):
    input_shape = (224, 224, 3)

    # Load the data
    train_images, train_labels, val_images, val_labels, age_encoder, gender_encoder, race_encoder = load_sample_data(
        data_path, input_shape, "train.csv", "val.csv", sample_percent=0.1
    )

    # Create the autoencoder model
    autoencoder = create_autoencoder(input_shape)
    autoencoder.compile(optimizer='adam', loss='mse')

    # Train the autoencoder
    autoencoder.fit(
        train_images, train_images,
        epochs=2,
        batch_size=32,
        validation_data=(val_images, val_images)
    )

    # Extract the encoder
    encoder = extract_encoder(autoencoder)

    # Create the classification model
    classification_model = create_classification_model(encoder, len(race_encoder.classes_), 
                                                       len(age_encoder.classes_), len(gender_encoder.classes_))

    # Compile the classification model
    classification_model.compile(optimizer='adam',
                                  loss={'race_output': 'sparse_categorical_crossentropy',
                                        'age_output': 'sparse_categorical_crossentropy',
                                        'gender_output': 'sparse_categorical_crossentropy'},
                                  metrics=['accuracy'])

    # Train the classification model
    classification_model.fit(train_images, {'race_output': train_labels[:, 2], 'age_output': train_labels[:, 0],
                                            'gender_output': train_labels[:, 1]},
                             epochs=10,
                             batch_size=32,
                             validation_data=(val_images, {'race_output': val_labels[:, 2],
                                                           'age_output': val_labels[:, 0],
                                                           'gender_output': val_labels[:, 1]}))

    # Predict the labels for the validation set
    val_predictions = classification_model.predict(val_images)
    val_pred_classes = [np.argmax(predictions, axis=-1) for predictions in val_predictions]

    # Compute the accuracy for each attribute
    print("Validation accuracy:")
    print("Age:")
    print(classification_report(val_labels[:, 0], val_pred_classes[1], target_names=age_encoder.classes_))
    print("Gender:")
    print(classification_report(val_labels[:, 1], val_pred_classes[2], target_names=gender_encoder.classes_))
    print("Race:")
    print(classification_report(val_labels[:, 2], val_pred_classes[0], target_names=race_encoder.classes_))

    return encoder

In [ ]:
data_path = "/Users/harshvardhan/Library/CloudStorage/Dropbox/Academics/UTK Classes/Spring 2023/Deep Learning/Final Project - FairFace Data/cosc-525-final-project/fairface-img-margin025-trainval"
run_model_dl(data_path)
